In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import fastai
fastai.__version__

# Extracting Data

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", 'r') as zip_ref:
                zip_ref.extractall("./")

with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", 'r') as zip_ref:
                zip_ref.extractall("./")

In [ ]:
from fastai.vision.all import *

# Data Validation

In [ ]:
path = Path('./')
path

In [ ]:
path.ls()

In [ ]:
fname = (path/"train").ls()[0]
fname.name

In [ ]:
pattern = r'(.+)\.\d+.jpg$'

In [ ]:
import re
re.findall(pattern, fname.name)

In [ ]:
from PIL import Image
image = Image.open(fname)
image.shape

# Building Data Loaders

In [ ]:
dblock = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter= RandomSplitter(seed=42),
    get_y = using_attr(RegexLabeller(pattern), 'name'),
    item_tfms=Resize(248),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)                   
)

In [ ]:
dataloaders = dblock.dataloaders(path/"train")

In [ ]:
dataloaders.show_batch(nrows=3, ncols=5)

# Model Building & Training

In [ ]:
learn = cnn_learner(dataloaders, resnet50, metrics=error_rate)

In [ ]:
learn.fine_tune(1)

# Results Validation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
learn.lr_find()

In [ ]:
test_images = (path/"test1")

In [ ]:
images = get_image_files(test_images)

In [ ]:
images[0]

In [ ]:
tst_dl = dataloaders.test_dl(images)

In [ ]:
tst_dl.show_batch()

In [ ]:
p = learn.get_preds(dl=tst_dl)

In [ ]:
type(p)

In [ ]:
p[0]

In [ ]:
p[1]

In [ ]:
image_paths = list(image_paths)

In [ ]:
??learn.get_preds

In [ ]:
for index in range(len(image_paths)):
    result = dict()
    im_path = image_paths[index]
    preds = learn.predict(im_path)[0]
    result['id'] = im_path
    result['target'] = preds  
    results.append(result)